In [ ]:
'''
Unit 7.6 Data Wrangling for Capstone 2
Nantawat Samermit 
'''

In [3]:
#import relevant modules for data wrangling - keep on adding as needed
import pandas as pd
import json, requests
import fuzzywuzzy 
import matplotlib

In [4]:
#ask Luka: Should I just use requests or should I use the API given by the data set manager?
#here is a code snippet from the API doc
#the source is: https://dev.socrata.com/foundry/data.cambridgema.gov/ayt4-g2ye

#from sodapy import Socrates
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cambridgema.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cambridgema.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
#results = client.get("ayt4-g2ye", limit=2000)

# Convert to pandas DataFrame
#results_df = pd.DataFrame.from_records(results)

In [5]:
#Cambridge MA wastewater dataset as JSON
url = 'https://data.cambridgema.gov/resource/ayt4-g2ye.json'

In [6]:
request = requests.get(url)
json_data = request.json()
json_data[0]

{'date': '2020-10-01T00:00:00.000',
 'mwra_concentration': '',
 'mwra_7dayaverage': '31',
 'concentration_mid_cambridge': '',
 'concentration_east_cambridge': '',
 'concentration_cambridgeport': '',
 'concentration_north_west': '',
 'highci_mid_cambridge': '',
 'highci_east_cambridge': '',
 'highci_cambridgeport': '',
 'highci_north_west_cambridge': '',
 'lowci_mid_cambridge': '',
 'lowci_east_cambridge': '',
 'lowci_cambridgeport': '',
 'lowci_north_west_cambridge': ''}

In [6]:
#confirming json_data is NOT a df yet
type(json_data)

list

In [7]:
#converting json_data to df
Cambridge_df = pd.DataFrame(json_data)

In [8]:
type(Cambridge_df)

pandas.core.frame.DataFrame

In [9]:
Cambridge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   date                          253 non-null    object
 1   mwra_concentration            253 non-null    object
 2   mwra_7dayaverage              253 non-null    object
 3   concentration_mid_cambridge   253 non-null    object
 4   concentration_east_cambridge  253 non-null    object
 5   concentration_cambridgeport   253 non-null    object
 6   concentration_north_west      253 non-null    object
 7   highci_mid_cambridge          253 non-null    object
 8   highci_east_cambridge         253 non-null    object
 9   highci_cambridgeport          253 non-null    object
 10  highci_north_west_cambridge   253 non-null    object
 11  lowci_mid_cambridge           253 non-null    object
 12  lowci_east_cambridge          253 non-null    object
 13  lowci_cambridgeport 

In [10]:
Cambridge_df.describe()

,date,mwra_concentration,mwra_7dayaverage,concentration_mid_cambridge,concentration_east_cambridge,concentration_cambridgeport,concentration_north_west,highci_mid_cambridge,highci_east_cambridge,highci_cambridgeport,highci_north_west_cambridge,lowci_mid_cambridge,lowci_east_cambridge,lowci_cambridgeport,lowci_north_west_cambridge
count,253,253,253,253,253,253,253,253,253,253,253,253,253,253,253
unique,253,216,197,28,30,24,30,28,30,24,30,28,30,24,30
top,2021-01-05T00:00:00.000,,257,,,,,,,,,,,,
freq,1,10,5,226,224,228,224,226,224,228,224,226,224,228,224


In [11]:
'''
For my own review!

Column Definitions:
Date - sample date for 24 hour composite sample
MWRA_Concentration - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
MWRA_7DayAverage - 7 day rolling average for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
Concentration_* - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
highci_* - Upper Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
lowci_* - Lower Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
* means there are multiple columns with this prefix which share the same definition - Nantawat

A refresher on confidence interval limits - https://bit.ly/3vtuI7h
'''

'\nFor my own review!\n\nColumn Definitions:\nDate - sample date for 24 hour composite sample\nMWRA_Concentration - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nMWRA_7DayAverage - 7 day rolling average for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nConcentration_* - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nhighci_* - Upper Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nlowci_* - Lower Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\n* means there are multiple columns with this prefix which share the same definition - Nantawat\n\nA refresher on confidence interval limits - https://bit.ly/3vtuI7h\n'

In [12]:
Cambridge_df.tail()

,date,mwra_concentration,mwra_7dayaverage,concentration_mid_cambridge,concentration_east_cambridge,concentration_cambridgeport,concentration_north_west,highci_mid_cambridge,highci_east_cambridge,highci_cambridgeport,highci_north_west_cambridge,lowci_mid_cambridge,lowci_east_cambridge,lowci_cambridgeport,lowci_north_west_cambridge
248,2021-06-06T00:00:00.000,24,25,,,,,,,,,,,,
249,2021-06-07T00:00:00.000,29,27,,,,,,,,,,,,
250,2021-06-08T00:00:00.000,15,24,,,,,,,,,,,,
251,2021-06-09T00:00:00.000,14,22,,,,,,,,,,,,
252,2021-06-10T00:00:00.000,33,23,,,,,,,,,,,,


In [13]:
#Note: There seems to be plenty of empty values, must make sure to sanitize them to NaN 
# The last date entry is for 6/10/21

In [14]:
#scraping for Massachusett's DoH dataset. Note this dataset's last entry is for 6/15/21
url2 = 'https://www.mass.gov/doc/covid-19-raw-data-june-15-2021/download'

In [38]:
#This seems like a big file, can I stream it in chunks?
Mass_request = requests.get(url2)    

In [2]:
#Note: Mass_request.content reveals something funky, can't describe it
#Mass_request.content

In [44]:
#from this StackOverflow article, BytesIO might be able to help
# https://stackoverflow.com/questions/58147689/python-how-to-read-excel-file-from-requests-response
import io

In [47]:
with io.BytesIO(Mass_request.content) as file:
    df = pd.io.excel.read_excel(file)

In [50]:
df.head()

,Current Table Name,Old Table Name,Current Variable Name,Old Variable Name,Frequency of Data Updates,Description,Notes,Unnamed: 7
0,Age MeansLast2Weeks,Age MeansLast2Weeks,Date,Date,Weekly,Date of when this data was reported,NaN,NaN
1,NaN,NaN,Start Date,NaN,NaN,"First day of the ""last two weeks"" period to wh...",NaN,NaN
2,NaN,NaN,End Date,NaN,NaN,"Last day of the ""last two weeks"" period to whi...",NaN,NaN
3,NaN,NaN,Mean Overall Age_Last2Weeks,Mean Overall Age_Last2Weeks,NaN,Date to which this data applies – by the date ...,NaN,NaN
4,NaN,NaN,Mean Hospitalized Age_Last2Weeks,Mean Hospitalized Age_Last2Weeks,NaN,Mean age of all probable and confirmed COVID-1...,NaN,NaN


In [ ]:
#Note: df does not match the excel spreadsheet I downloaded, something went wrong. 
#Maybe the url2 request didn't grab the .xls
#Checked in my chrome downloads folder, the .xls should be around 2,425 kb. Small enough for bs4?

In [15]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

In [63]:
#Request is forbidden 403 - try BS4
#url2 = 'https://www.mass.gov/doc/covid-19-raw-data-june-15-2021/download'
#request = Request(url2)
#response = urlopen(request)
#print(type(response))
#response.close()

HTTPError: HTTP Error 403: Forbidden

In [64]:
#Once again, the file is too big! Must consider streaming in chunks
#r = requests.get(url2)
#text = r.text
#print(text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
#found a link to the specific xls
#testing to see if BS4 can ID and isolate the href with .xls 
url3 = 'https://www.mass.gov/doc/covid-19-raw-data-june-15-2021'
r = requests.get(url3)
html_doc = r.text
soup = BeautifulSoup(html_doc)
print(soup.title)

<title>covid-19-raw-data-file-6-15-2021.xlsx | Mass.gov</title>


In [20]:
file_format = '.xlsx'
a_tags = soup.find_all('a')

for link in a_tags:
    print(link.get('href'))

#main-content
/info-details/massachusetts-state-organizations-a-to-z
https://sso.hhs.state.ma.us/
https://uionline.detma.org/Claimant/Core/Login.ASPX
https://ecse.cse.state.ma.us/ECSE/Login/login.asp
/
/topics/health-social-services
/topics/families-children
/topics/housing-property
/topics/transportation
/topics/legal-justice
/topics/public-safety
/topics/energy
/topics/environment
/topics/taxes
/topics/unclaimed-property
/topics/vital-public-records
/topics/voting
/topics/living
/topics/business-resources
/topics/professional-licenses-permits
/topics/unemployment
/topics/finding-a-job
/topics/taxes
/topics/workers-rights-safety
/topics/industry-regulations
/topics/state-employee-resources
/topics/working
/topics/early-childhood-education-care
/topics/elementary-secondary-schools
/topics/higher-education
/topics/continuing-education
/topics/for-educators-administrators
/topics/professional-training-career-development
/topics/learning
/topics/parks-recreation
/topics/fishing-hunting
/t

In [ ]:
#Stuck here. It seems the xls href doesn't point to the file itself, but maybe to a webapp /download
#which obfuscates the file location. Ask Luka